## Python Demo

In [1]:
# this just changes the size of the cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

### Email!

In [2]:
# EMAIL
import  win32com.client as win32

outlook = win32.Dispatch("Outlook.Application").GetNamespace("MAPI") # connect
folder = outlook.Folders('Survey Support') # get a folder
folders = folder.folders # get the folders in that folder
inbox = folders('Inbox') # get a folder in that folder
msgs = inbox.Items # get the stuff in that folder

com_error: (-2147352567, 'Exception occurred.', (4096, 'Microsoft Outlook', 'Cannot complete the operation. You are not connected.', None, 0, -2147352567), None)

In [ ]:
# that is a pretty simple for loop..

[msg.subject for msg in msgs]

### Folders!

In [4]:
# whats in me folder?
import os

os.chdir(r'C:\Users\Mike.Hamilton\Desktop') # change directory
[f for f in os.listdir()] # list directory and loop through it

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Mike.Hamilton\\Desktop'

In [ ]:
# Whats in all the folders!
fldr = r'C:\Users\Mike.Hamilton\Desktop'
[f[2] for f in os.walk(fldr)]

In [ ]:
# Open a file
with open (r'C:\Users\Mike.Hamilton\Desktop\Summary.txt', 'r') as f: # with is an important 'context manager' it ensures we are closing files
    file = f.readlines()

In [ ]:
# have a look at it
[l for l in file] # where's the bit where I say what l has to be eh??

In [ ]:
# Random manipulation & extract 

file = [l.split(',') for l in file] # split up the lines by comma
file = [(l[1][:10], l[4]) for l in file] # get the 1st ten characters of the 2nd item and the 5th item of each line 
file

In [ ]:
with open(r'C:\Users\Mike.Hamilton\Desktop\Hokey Demo.txt', 'w') as wr:
    for l in file:
        wr.write(str(l)) # as we have a list of tuples we either dial in to the tuple or join it or string it
        wr.write('\n') # the joy of new lines

### Zipfiles!

In [ ]:
import os
from zipfile import ZipFile

os.chdir('C:/Users/Mike.Hamilton/Desktop/test zip')
fldr = os.getcwd()

In [ ]:
# Let's zip everything up! - note the commented out print statements...
for f in os.listdir(fldr):
    # print(f)
    with ZipFile(f + '.zip', 'w') as zipObj:
       # Iterate over all the files in directory
       for folderName, subfolders, filenames in os.walk(f):
            for filename in filenames:
                # print(foldername, filename)
                # create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # print(filePath)
                # Add file to zip
                zipObj.write(filePath)

### Database!

In [ ]:
# let's Connect!
import pyodbc # db connecting module conforms to the python db api - as does psycopg2 (postgress)

server = "ldnsasql1" # that's us
data_base = "YW_3" # that's them
conn_str = 'Driver={SQL Server};Server='+server+';Database='+data_base+';Trusted_Connection=yes;'
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

In [ ]:
# Some mildly random variables
tbl = 'ImportRawData'
sltc = 'rawdata'
filt = 'ImportFileID'
x = '24' 
scripty = 'SELECT {} from {} where {} = {}'.format(sltc, tbl, filt, x) # stick em in an  sql srting

In [ ]:
scripty

In [ ]:
cursor.execute(scripty) # our cursor is now loaded with the result of our script
tbl = [l[0] for l in cursor] # this will empty our cursor
tbl[-10:] # the last 10 lines of our cursor output

### Trello?

In [ ]:
import trello

TRELLO_APP_KEY = '97d852ec5cff3da4652e772751325592' # get your own!!
from trello import TrelloApi
trello = TrelloApi(TRELLO_APP_KEY)
tok = '82520d3db47c3ee5697b99475df9866b38aa009b4518cea9d5a432eccabe03fe'
trello.set_token(tok)

In [ ]:
boards = trello.members.get_board('mikehamilton56')

In [ ]:
for b in boards:
    print('{} - {} - {}'.format(b['name'],b['desc'], b['id']) )

In [ ]:
for b in boards[-3:]:
    cards = trello.boards.get_card(b['id'])
    for c in cards[:10]:
#        print(c)
        print('{}, Card {}, Due {}'.format(b['name'], c['name'], c['due'] ))

### OMG Survey Gizmo - thanks Craig!

In [ ]:
import requests

#Set up Authentication for API
#Important Keys
client_key = '3fc719ae2434533e979d1764675168a5796965ebc6ffxxxxx'
client_secret = r'A9gG/ssSBxxxxx'
API_Auth = '?api_token=' + client_key + '&api_token_secret=' + client_secret

#API Strings
API_Survey = 'http://restapi.surveygizmo.eu' #Really Important this points to eu server
API_Version = 'v4'
API_Object = 'survey'
API_SubObject = 'surveyresponse'
survey_ID = 90179090 

#Make API Request
API_String = API_Survey + '/' + API_Version + '/' + API_Object + '/' + str(survey_ID) + '/' + API_SubObject + API_Auth + '&resultsperpage=500'
response = requests.get(API_String)

#Convert JSON response into top level Dict
response_lvl1 = response.json()
#Extract actual data
response_lvl2 = response_lvl1['data']
#See if there is more than one page
total_pages = response_lvl1['total_pages']
#Loop through remaining pages
if total_pages > 1:
    for pageNum in range(2, total_pages + 1):
        #Repeat Request for next page
        API_Page_String = API_String + '&page=' + str(pageNum)
        #print(API_Page_String)
        response = requests.get(API_Page_String)
        if response.status_code == 200:
            #Convert JSON response into top level Dict
            response_page = response.json()
            #Extract actual data
            response_lvl2 = response_lvl2 + response_page['data']

In [ ]:
# this is the JSON data
response_lvl2[0]

#### We'll do a bit of messing about

In [ ]:
# get all the results of a certain key

Locs = [item['[variable("STANDARD_GEOCITY")]'] for item in response_lvl2]
Locs

In [ ]:
# A simple pivot

from collections import Counter
Locs_Tots = list(Counter(Locs).items())
Locs_Tots

In [ ]:
# and a sorted top ten
Locs_Top_20 = sorted(Locs_Tots, key=lambda x: x[1], reverse=True)[:20]
Locs_Top_20

### A Very Amazing Chart

In [5]:
os.chdir(r'C:\Users\Mike.Hamilton\Desktop')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Mike.Hamilton\\Desktop'

In [6]:
from matplotlib import pyplot as plt
ldg = [l[0] for l in Locs_Top_20]
vls = [l[1] for l in Locs_Top_20]
plt.figure(figsize=(15, 8))
plt.xticks(color='deeppink', rotation=90)
plt.subplots_adjust(bottom= 0.3, top = 0.95)
plt.bar(ldg, vls, width=.75, color='magenta')
plt.gca().set_title('Amazing Bar Chart')
plt.gca().set_facecolor('cyan')
plt.yticks(color='violet')
plt.savefig('the_best_plot_eva.pdf')

NameError: name 'Locs_Top_20' is not defined

### this chart I got off the internet is not as good as the one I did above in like less than a day and all by myself with no ones help

In [7]:
import seaborn as sns
import pandas as pd
 
# Data set
url = 'https://python-graph-gallery.com/wp-content/uploads/mtcars.csv'
df = pd.read_csv(url)
df = df.set_index('model')
del df.index.name
df
 
# # this bit is the all that it takes to make the chart is bit
sns.clustermap(df, z_score=1)


ModuleNotFoundError: No module named 'seaborn'

In [8]:
son_of_scripty = r'''
SELECT[QuestionID]
      ,coalesce([Response1], 0)
      ,coalesce([Response2], 0)
      ,coalesce([Response3], 0)
      ,coalesce([Response4], 0)
      ,coalesce([Response5], 0)
    FROM [AggregateQuestion]
where
coalesce([Response1], 0)
      +coalesce([Response2], 0)
      +coalesce([Response3], 0)
      +coalesce([Response4], 0)
      +coalesce([Response5], 0) > 2000
'''


In [ ]:
cursor.execute(son_of_scripty)

In [ ]:
tbl = [str(l)[1:-1].split(',') for l in cursor]
tbl = [[int(x) for x in l] for l in tbl]

In [ ]:
tbl

In [ ]:
df2 = pd.DataFrame(tbl, columns = ['QuestionID','Response1','Response2',\
                                   'Response3','Response4','Response5'])
df2 = df2.set_index('QuestionID')
del df2.index.name

In [ ]:
df2

In [ ]:
sns.clustermap(df2)

In [ ]:
sns.set()

# Load the example flights dataset and conver to long-form
flights_long = sns.load_dataset("flights")
flights = flights_long.pivot("month", "year", "passengers")

# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(flights, annot=True, fmt="d", linewidths=.5, ax=ax)
